In [1]:
import os
import json
import numpy as np
import pandas as pd
import os.path as op
from sklearn.metrics import roc_auc_score

In [2]:
files = [a for a in os.walk("../results/")][0][2]

In [3]:
def get_data(x, f):
    with open(op.join(f, x), "r") as oh:
        return(json.load(oh))

In [4]:
def aurocs(a):
    return(
        [
            roc_auc_score(y_true=a["train.reals"], y_score=a["train.preds"]),
            roc_auc_score(y_true=a["test.reals"], y_score=a["test.preds"])
        ]
    )

In [6]:
files = list(filter(lambda x: ".json" in x and "test1" not in x, files))

In [7]:
ru_1hot = list(
    sorted(
        list(filter(lambda x: "_1hot_" in x and "ru" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
en_1hot = list(
    sorted(
        list(filter(lambda x: "_1hot_" in x and "en" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
ru_wv = list(
    sorted(
        list(filter(lambda x: "_wv_" in x and "ru" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
en_wv = list(
    sorted(
        list(filter(lambda x: "_wv_" in x and "en" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
ru_1hot = [get_data(a, "../results/") for a in ru_1hot]
en_1hot = [get_data(a, "../results/") for a in en_1hot]
ru_wv = [get_data(a, "../results/") for a in ru_wv]
en_wv = [get_data(a, "../results/") for a in en_wv]

In [8]:
ru_sham_aucs_1hot = [aurocs(a) for a in ru_1hot[0:23]]
ru_vl_aucs_1hot = [aurocs(a) for a in ru_1hot[24:23+25+1]]
ru_eoff_aucs_1hot = [aurocs(a) for a in ru_1hot[23+25+1:23+25+25]]
ru_eon_aucs_1hot = [aurocs(a) for a in ru_1hot[23+25+25:]]
ru_sham_aucs_wv = [aurocs(a) for a in ru_wv[0:23]]
ru_vl_aucs_wv = [aurocs(a) for a in ru_wv[24:23+25+1]]
ru_eoff_aucs_wv = [aurocs(a) for a in ru_wv[23+25+1:23+25+25]]
ru_eon_aucs_wv = [aurocs(a) for a in ru_wv[23+25+25:]]

In [9]:
en_sham_aucs_1hot = [aurocs(a) for a in en_1hot[0:17]]
en_vl_aucs_1hot = [aurocs(a) for a in en_1hot[17:]]
en_sham_aucs_wv = [aurocs(a) for a in en_wv[0:17]]
en_vl_aucs_wv = [aurocs(a) for a in en_wv[17:]]

In [10]:
ru_1hot_aucs = np.stack([aurocs(a) for a in ru_1hot])
en_1hot_aucs = np.stack([aurocs(a) for a in en_1hot])
ru_wv_aucs = np.stack([aurocs(a) for a in ru_wv])
en_wv_aucs = np.stack([aurocs(a) for a in en_wv])

In [11]:
def get_results(total, sham, vl, eoff, eon):
    return(
        pd.DataFrame(
            {
                "trial": [
                    "all", "sham", "vl", "eoff", "eon"
                ],
                "train_mean": [
                    np.mean(total, 0)[0], np.mean(sham, 0)[0],
                    np.mean(vl, 0)[0], np.mean(eoff, 0)[0],
                    np.mean(eon, 0)[0]
                ],
                "train_std": [
                    np.std(total, 0)[0], np.std(sham, 0)[0],
                    np.std(vl, 0)[0], np.std(eoff, 0)[0],
                    np.std(eon, 0)[0]
                ],
                "test_mean": [
                    np.mean(total, 0)[1], np.mean(sham, 0)[1],
                    np.mean(vl, 0)[1], np.mean(eoff, 0)[1],
                    np.mean(eon, 0)[1]
                ],
                "test_std": [
                    np.std(total, 0)[1], np.std(sham, 0)[1],
                    np.std(vl, 0)[1], np.std(eoff, 0)[1],
                    np.std(eon, 0)[1]
                ]
            }
        )
    )

In [12]:
ru_1hot_results = get_results(
    ru_1hot_aucs, ru_sham_aucs_1hot, ru_vl_aucs_1hot, ru_eoff_aucs_1hot,
    ru_eon_aucs_1hot
)
ru_wv_results = get_results(
    ru_wv_aucs, ru_sham_aucs_wv, ru_vl_aucs_wv, ru_eoff_aucs_wv,
    ru_eon_aucs_wv
)
en_1hot_results = get_results(
    en_1hot_aucs, en_sham_aucs_1hot, en_vl_aucs_1hot, en_vl_aucs_1hot,
    en_sham_aucs_1hot
).iloc[0:3]
en_wv_results = get_results(
    en_wv_aucs, en_sham_aucs_wv, en_vl_aucs_wv, en_vl_aucs_wv,
    en_sham_aucs_wv
).iloc[0:3]

In [13]:
ru_1hot_results

,trial,train_mean,train_std,test_mean,test_std
0,all,0.499372,0.007430,0.496418,0.018047
1,sham,0.499064,0.005985,0.499588,0.016411
2,vl,0.500612,0.002076,0.498390,0.005480
3,eoff,0.500626,0.002863,0.500278,0.001332
4,eon,0.497553,0.012083,0.489136,0.028442


In [14]:
print(ru_1hot_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &    0.499372 &   0.007430 &   0.496418 &  0.018047 \\
1 &  sham &    0.499064 &   0.005985 &   0.499588 &  0.016411 \\
2 &    vl &    0.500612 &   0.002076 &   0.498390 &  0.005480 \\
3 &  eoff &    0.500626 &   0.002863 &   0.500278 &  0.001332 \\
4 &   eon &    0.497553 &   0.012083 &   0.489136 &  0.028442 \\
\bottomrule
\end{tabular}



In [15]:
ru_wv_results

,trial,train_mean,train_std,test_mean,test_std
0,all,1.0,0.0,0.514260,0.088831
1,sham,1.0,0.0,0.510418,0.084398
2,vl,1.0,0.0,0.501361,0.090922
3,eoff,1.0,0.0,0.539615,0.101536
4,eon,1.0,0.0,0.507257,0.076000


In [16]:
print(ru_wv_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &         1.0 &        0.0 &   0.514260 &  0.088831 \\
1 &  sham &         1.0 &        0.0 &   0.510418 &  0.084398 \\
2 &    vl &         1.0 &        0.0 &   0.501361 &  0.090922 \\
3 &  eoff &         1.0 &        0.0 &   0.539615 &  0.101536 \\
4 &   eon &         1.0 &        0.0 &   0.507257 &  0.076000 \\
\bottomrule
\end{tabular}



In [17]:
en_1hot_results

,trial,train_mean,train_std,test_mean,test_std
0,all,0.501861,0.008696,0.502409,0.009694
1,sham,0.503721,0.012014,0.504819,0.013279
2,vl,0.500000,0.000000,0.500000,0.000000


In [19]:
print(en_1hot_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &    0.501861 &   0.008696 &   0.502409 &  0.009694 \\
1 &  sham &    0.503721 &   0.012014 &   0.504819 &  0.013279 \\
2 &    vl &    0.500000 &   0.000000 &   0.500000 &  0.000000 \\
\bottomrule
\end{tabular}



In [20]:
en_wv_results

,trial,train_mean,train_std,test_mean,test_std
0,all,1.0,0.0,0.472489,0.073007
1,sham,1.0,0.0,0.470875,0.072524
2,vl,1.0,0.0,0.474104,0.073451


In [21]:
print(en_wv_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &         1.0 &        0.0 &   0.472489 &  0.073007 \\
1 &  sham &         1.0 &        0.0 &   0.470875 &  0.072524 \\
2 &    vl &         1.0 &        0.0 &   0.474104 &  0.073451 \\
\bottomrule
\end{tabular}

